In [1]:
import numpy as np
import os

In [11]:
base_script = """#!/bin/bash

#SBATCH --job-name={}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16GB
#SBATCH --time=100:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --output="o{}-%j.out"

module purge
module load anaconda3/4.3.1
module load sox/intel/14.4.2
module load ffmpeg/intel/3.2.2
module load cuda/10.1.105
module load cudnn/10.0v7.6.2.24

source activate py3.5

SLICES='{}'
MODELNAME='{}'

cd /home/ci411/pcen-t-varying/scripts/core_scripts

python 03_evaluate.py --slices=$SLICES --model-id=$MODELNAME --model-dir='/beegfs/ci411/pcen/models/models_122' --feature-dir='/beegfs/ci411/pcen/features/' --results-name='results_dry'"""

In [29]:
model_dir = '/beegfs/ci411/pcen/models/models_122'

models = [item for item in os.listdir(model_dir) if "model122" in item]

slices_list = [str([int(single_slice) for single_slice in slices.split('_')[2]]).replace(' ','') for slices in models]

In [30]:
slices_list

['[0,1,2]',
 '[2,3,4,5,6,7]',
 '[0,1,2,3]',
 '[0,1,2]',
 '[0,1]',
 '[0,1,2,3,4,5,6,7]',
 '[0]',
 '[0,1,2,3]',
 '[2,3,4,5,6,7]',
 '[0,1,2,3,4,5,6]',
 '[0,1,2,3,4,5,6,7,8]',
 '[4,5]',
 '[0,1,2,3,4,5,6,7,8]',
 '[4,5]',
 '[0]',
 '[0,1,2,3,4,5]',
 '[0,1]',
 '[1,2,3,4,5,6,7,8]',
 '[0,1,2,3,4,5,6,7]',
 '[0,1,2,3,4,5]',
 '[0,1,2,3,4]',
 '[0,1,2,3,4,5,6,7,8,9]',
 '[1,2,3,4,5,6,7,8]',
 '[3,4,5,6]',
 '[0,1,2,3,4]',
 '[3,4,5,6]',
 '[0,1,2,3,4,5,6]',
 '[0,1,2,3,4,5,6,7,8,9]']

In [34]:
model_names = [model.split('_')[1]+model.split('_')[2] for model in models]

In [35]:
model_names

['dry012',
 'wet234567',
 'dry0123',
 'wet012',
 'dry01',
 'wet01234567',
 'wet0',
 'wet0123',
 'dry234567',
 'wet0123456',
 'dry012345678',
 'dry45',
 'wet012345678',
 'wet45',
 'dry0',
 'dry012345',
 'wet01',
 'wet12345678',
 'dry01234567',
 'wet012345',
 'dry01234',
 'wet0123456789',
 'dry12345678',
 'wet3456',
 'wet01234',
 'dry3456',
 'dry0123456',
 'dry0123456789']

In [38]:
base_script.format('name', 'name', 'slices', 'modelname')

'#!/bin/bash\n\n#SBATCH --job-name=name\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=4\n#SBATCH --mem=16GB\n#SBATCH --time=100:00:00\n#SBATCH --mail-user=chris.ick@nyu.edu\n#SBATCH --output="oname-%j.out"\n\nmodule purge\nmodule load anaconda3/4.3.1\nmodule load sox/intel/14.4.2\nmodule load ffmpeg/intel/3.2.2\nmodule load cuda/10.1.105\nmodule load cudnn/10.0v7.6.2.24\n\nsource activate py3.5\n\nSLICES=\'slices\'\nMODELNAME=\'modelname\'\n\ncd /home/ci411/pcen-t-varying/scripts/core_scripts\n\npython 03_evaluate.py --slices=$SLICES --model-id=$MODELNAME --model-dir=\'/beegfs/ci411/pcen/models/models_122\' --feature-dir=\'/beegfs/ci411/pcen/features/\' --results-name=\'results_dry\''

In [39]:
#make scripts
for i, slices in enumerate(slices_list):
    model_abbrev = model_names[i]
    modelname = models[i]
    filename = modelname + '_dry_eval.sbatch'
    sbatch_str = base_script.format(model_abbrev, model_abbrev, slices, modelname)
    with open(filename, 'w+') as f:
        f.write(sbatch_str)

In [40]:
with open('runall.bat', 'w+') as f:
    for model in models:
        f.write('sbatch '+ model + '_dry_eval.sbatch' + '\n')